# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [2]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"
os.environ["USER_AGENT"] = "MyLangChainApp/1.0"

In [3]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [4]:
# TODO: Import traceable
from groq import Groq
from typing import List
from langsmith import traceable
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.3-70b-versatile"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

client = Groq()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_llm(messages)

@traceable
def call_llm(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


@traceable handles the RunTree lifecycle for you!

In [5]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the @traceable decorator, you need to import it from langsmith and decorate the function you want to trace. You also need to set the LANGCHAIN_TRACING_V2 environment variable to 'true' and the LANGCHAIN_API_KEY environment variable to your API key. The decorator can be used like this: @traceable or @traceable(run_type="retriever") to specify the run type.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [6]:
from langsmith import traceable

@traceable(
    # TODO: Add Metadata
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_llm(messages)

@traceable(
    # TODO: Add Metadata
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_llm(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [7]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To add metadata to a run with @traceable, you can pass an options object with a metadata property when calling the traceable function. However, the provided context does not include a specific example of how to do this. You can refer to the Concepts page for more information on metadata and tags. For more detailed instructions, you may need to consult the LangSmith documentation.


You can also add metadata at runtime!

In [8]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

LangSmith supports sending arbitrary metadata and tags along with traces, allowing you to associate additional information with a trace. You can add metadata to each example, such as a note or a source, which is stored as a dictionary. For more information on metadata and tags, see the Concepts page.


##### Let's take a look in LangSmith!